In [19]:
import numpy as np
import pandas as pd
from scipy.cluster import hierarchy
from scipy.spatial import distance_matrix
from sklearn.metrics.pairwise import euclidean_distances
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
%matplotlib inline

<hr>
<h1 id="clustering_vehicle_dataset">Clustering on Vehicle dataset</h1>

* Imaginez qu'un constructeur automobile développe des prototypes pour un nouveau véhicule. Avant d'introduire le nouveau modèle dans sa gamme, le constructeur souhaite déterminer quels véhicules existants sur le marché ressemblent le plus aux prototypes, c'est-à-dire comment les véhicules peuvent être regroupés, quel groupe est le plus similaire au modèle, et avec quels modèles ils seront en concurrence.

* Notre objectif ici est d'utiliser une méthode de regroupement pour trouver les groupes de véhicules les plus distinctifs. Il résumera les véhicules existants et aidera les constructeurs à prendre des décisions concernant la fourniture de nouveaux modèles.



## Chargement du dataset

* Lisons l'ensemble de données pour voir quelles fonctionnalités le fabricant a collectées sur les modèles existants.


In [20]:
filename = 'C:/Users/user/Desktop/Doc Professionnel/ODC/Veille 8/cars_clus.csv'

#Read csv
pdf = pd.read_csv(filename)
print ("Forme du dataset: ", pdf.shape)

pdf.head(5)

Forme du dataset:  (159, 16)


,manufact,model,sales,resale,type,price,engine_s,horsepow,wheelbas,width,length,curb_wgt,fuel_cap,mpg,lnsales,partition
0,Acura,Integra,16.919,16.360,0.000,21.500,1.800,140.000,101.200,67.300,172.400,2.639,13.200,28.000,2.828,0.0
1,Acura,TL,39.384,19.875,0.000,28.400,3.200,225.000,108.100,70.300,192.900,3.517,17.200,25.000,3.673,0.0
2,Acura,CL,14.114,18.225,0.000,$null$,3.200,225.000,106.900,70.600,192.000,3.470,17.200,26.000,2.647,0.0
3,Acura,RL,8.588,29.725,0.000,42.000,3.500,210.000,114.600,71.400,196.600,3.850,18.000,22.000,2.150,0.0
4,Audi,A4,20.397,22.255,0.000,23.990,1.800,150.000,102.600,68.200,178.000,2.998,16.400,27.000,3.015,0.0


In [21]:
pdf.partition.unique()

array([0., 1.])

* Les ensembles de fonctionnalités incluent le prix en milliers (prix), la taille du moteur (engine_s), la puissance (horsepow), l'empattement (wheelbas), la largeur (width), la longueur (length), le poids à vide (curb_wgt), la capacité de carburant (fuel_cap) et le carburant efficacité (mpg).

<h2 id="data_cleaning">Pretraitement</h2>

In [22]:
pdf[[ 'sales', 'resale', 'type', 'price', 'engine_s',
       'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']] = pdf[['sales', 'resale', 'type', 'price', 'engine_s',
       'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']].apply(pd.to_numeric, errors='coerce')

In [23]:
pdf.isna().sum()

manufact      2
model         0
sales         2
resale       38
type          2
price         2
engine_s      1
horsepow      1
wheelbas      1
width         1
length        1
curb_wgt      2
fuel_cap      1
mpg           3
lnsales       2
partition     0
dtype: int64

In [24]:
features = pdf[[ 'sales', 'resale', 'type', 'price', 'engine_s',
       'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']]

In [25]:
for i in features.columns:
    features[i].replace(np.nan, features[i].mean(), inplace = True)

C:\Users\user\AppData\Local\Temp\ipykernel_6584\1463091702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[i].replace(np.nan, features[i].mean(), inplace = True)


In [26]:
features.isnull().sum()

sales       0
resale      0
type        0
price       0
engine_s    0
horsepow    0
wheelbas    0
width       0
length      0
curb_wgt    0
fuel_cap    0
mpg         0
lnsales     0
dtype: int64

In [27]:
pdf['manufact'].mode()

0    Dodge
1     Ford
Name: manufact, dtype: object

In [28]:
pdf['manufact'].replace(np.nan, 'Dodge', inplace = True)

In [29]:
features.replace(np.nan, )

,sales,resale,type,price,engine_s,horsepow,wheelbas,width,length,curb_wgt,fuel_cap,mpg,lnsales
0,16.919000,16.360000,0.000000,21.500000,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2.828000
1,39.384000,19.875000,0.000000,28.400000,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,3.673000
2,14.114000,18.225000,0.000000,27.396605,3.2,225.0,106.9,70.6,192.0,3.470,17.2,26.0,2.647000
3,8.588000,29.725000,0.000000,42.000000,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,2.150000
4,20.397000,22.255000,0.000000,23.990000,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,3.015000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,17.531000,18.072975,0.000000,28.800000,2.4,168.0,104.9,69.3,186.2,3.259,17.9,25.0,2.864000
155,3.493000,18.072975,0.000000,45.500000,2.3,236.0,104.9,71.5,185.7,3.601,18.5,23.0,1.251000
156,18.969000,18.072975,0.000000,36.000000,2.9,201.0,109.9,72.1,189.8,3.600,21.1,24.0,2.943000
157,52.998076,18.072975,0.261146,21.500000,1.5,76.0,106.3,67.9,175.0,2.932,11.9,46.0,3.295847


In [30]:
pdf = pdf.dropna()
pdf = pdf.reset_index(drop=True)
print ("Après traitement: ", pdf.size)
pdf.head(5)

Après traitement:  1872


,manufact,model,sales,resale,type,price,engine_s,horsepow,wheelbas,width,length,curb_wgt,fuel_cap,mpg,lnsales,partition
0,Acura,Integra,16.919,16.360,0.0,21.50,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2.828,0.0
1,Acura,TL,39.384,19.875,0.0,28.40,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,3.673,0.0
2,Acura,RL,8.588,29.725,0.0,42.00,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,2.150,0.0
3,Audi,A4,20.397,22.255,0.0,23.99,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,3.015,0.0
4,Audi,A6,18.780,23.555,0.0,33.95,2.8,200.0,108.7,76.1,192.0,3.561,18.5,22.0,2.933,0.0


In [31]:
pdf.drop(pdf[['sales', 'resale', 'type', 'price', 'engine_s','horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap',
       'mpg', 'lnsales']], axis=1, inplace = True)
print(pdf)

       manufact    model  partition
0         Acura  Integra        0.0
1         Acura       TL        0.0
2         Acura       RL        0.0
3          Audi       A4        0.0
4          Audi       A6        0.0
..          ...      ...        ...
112  Volkswagen     Golf        0.0
113  Volkswagen    Jetta        0.0
114  Volkswagen   Passat        0.0
115  Volkswagen   Cabrio        0.0
116  Volkswagen      GTI        0.0

[117 rows x 3 columns]


In [32]:
pdf = pd.concat([pdf, features], axis = 1)
pdf.head()

,manufact,model,partition,sales,resale,type,price,engine_s,horsepow,wheelbas,width,length,curb_wgt,fuel_cap,mpg,lnsales
0,Acura,Integra,0.0,16.919,16.360,0.0,21.500000,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2.828
1,Acura,TL,0.0,39.384,19.875,0.0,28.400000,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,3.673
2,Acura,RL,0.0,14.114,18.225,0.0,27.396605,3.2,225.0,106.9,70.6,192.0,3.470,17.2,26.0,2.647
3,Audi,A4,0.0,8.588,29.725,0.0,42.000000,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,2.150
4,Audi,A6,0.0,20.397,22.255,0.0,23.990000,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,3.015


In [33]:
pdf.type

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
154    0.000000
155    0.000000
156    0.000000
157    0.261146
158    0.261146
Name: type, Length: 159, dtype: float64

### Selection des fonctionnalités

In [34]:
featureset = features[['engine_s',  'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap', 'mpg']]

In [35]:
featureset.isna().sum()

engine_s    0
horsepow    0
wheelbas    0
width       0
length      0
curb_wgt    0
fuel_cap    0
mpg         0
dtype: int64

### Normalisation

* Nous utilisons **MinMaxScaler** pour mettre nos featureset sur une plage de (0, 1) 


In [36]:
x = featureset.values #returns a numpy array
min_max_scaler = MinMaxScaler()
feature_mtx = min_max_scaler.fit_transform(x)
feature_mtx [0:5]

NameError: name 'MinMaxScaler' is not defined

In [37]:
feature_mtx.shape

NameError: name 'feature_mtx' is not defined

<h2 id="clustering_using_skl">Regroupement</h2>


* Nous calculons la matrice de distance

In [ ]:
dist_matrix = euclidean_distances(feature_mtx, feature_mtx) 
dist_matrix.shape

In [ ]:
Z_using_dist_matrix = hierarchy.linkage(dist_matrix, method = 'complete')
Z_using_dist_matrix.shape

In [ ]:
fig = plt.figure(figsize=(18,50))
def llf(id):
    return '[%s %s %s]' % (pdf['manufact'][id], pdf['model'][id], int(float(pdf['type'][id])) )
    
dendro = hierarchy.dendrogram(Z_using_dist_matrix,  leaf_label_func=llf, leaf_rotation=0, leaf_font_size =12, orientation = 'right')

In [ ]:
dist_matrix.shape

* Nous utilisons **AgglomerativeClustering** de la bibliothèque scikit-learn pour regrouper l'ensemble de donnée par cluster

In [ ]:
agglom = AgglomerativeClustering(n_clusters = 6, linkage = 'complete')
agglom.fit(dist_matrix)

agglom.labels_

* Nous pouvons ajouter un nouveau champ à notre dataframe pour afficher le cluster de chaque ligne

In [ ]:
pdf['cluster_'] = agglom.labels_
pdf.head()

In [ ]:
pdf['cluster_'].unique()

In [ ]:
# max(agglom.labels_)

In [ ]:
import matplotlib.cm as cm
n_clusters = max(agglom.labels_)+1
colors = cm.rainbow(np.linspace(0, 1, n_clusters))

cluster_labels = list(range(0, n_clusters))

plt.figure(figsize=(16,14))

for color, label in zip(colors, cluster_labels):
    subset = pdf[pdf.cluster_ == label]
#     for i in subset.index:
#         plt.text(subset.horsepow[i], subset.mpg[i],str(subset['model'][i]), rotation=25) 
    plt.scatter(subset.horsepow, subset.mpg, s= subset.price*10, c=color, label='cluster'+str(label),alpha=0.5)
    plt.scatter(subset.horsepow, subset.mpg)
plt.legend()
plt.title('Clusters')
plt.xlabel('horsepow')
plt.ylabel('mpg');

* Comme vous pouvez le voir, nous voyons la distribution de chaque cluster à l'aide du nuage de points, mais il n'est pas très clair où se trouve le centroïde de chaque cluster. De plus, il existe 2 types de véhicules dans notre jeu de données, "camion" (valeur de 1 dans la colonne type) et "voiture" (valeur de 0 dans la colonne type). Nous les utilisons donc pour distinguer les classes et résumer le cluster. Nous comptons d'abord le nombre de cas dans chaque groupe :


In [ ]:
pdf.groupby(['cluster_','type'])['cluster_'].count()

Nous pouvons maintenant examiner les caractéristiques de chaque cluster 

In [ ]:
agg_cars = pdf.groupby(['cluster_','type'])['horsepow','engine_s','mpg','price'].mean()

agg_cars

Nous avons 3 clusters principaux avec la majorité des véhicules dans ceux-ci.

**Voitures**:

* Groupe 1 : avec un mpg presque élevé et une faible puissance.

* Groupe 2 : avec un bon mpg et une bonne puissance, mais un prix plus élevé que la moyenne.

* Groupe 3 : avec un faible mpg, une puissance élevée, le prix le plus élevé.

**Camions** :

* Groupe 1 : avec le mpg presque le plus élevé parmi les camions, et le plus bas en puissance et en prix.
* Groupe 2 : avec un mpg presque bas et une puissance moyenne, mais un prix plus élevé que la moyenne.
* Groupe 3 : avec un bon mpg et une bonne puissance, à bas prix.



In [ ]:
plt.figure(figsize=(16,10))
for color, label in zip(colors, cluster_labels):
    subset = agg_cars.loc[(label,),]
    for i in subset.index:
        plt.text(subset.loc[i][0]+5, subset.loc[i][2], 'type='+str(int(i)) + ', price='+str(int(subset.loc[i][3]))+'k')
    plt.scatter(subset.horsepow, subset.mpg, s=subset.price*20, c=color, label='cluster'+str(label))
plt.legend()
plt.title('Clusters')
plt.xlabel('horsepow')
plt.ylabel('mpg');